In [1]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

After a couple hours of trying everything I could think of, the following series of commands are the ONLY way I could figure out how to get data from the DataSocket to update. Specifically, The correct ReadMode, AccessMode must be used, and you must NOT call DataSocket.Update(). 

Other attempts resulted in complete failure to obtain data, obtain data only on 1st call, and in a couple places stalled the working thread. 

In [2]:
try:
    ds.SyncDisconnect(-1)
except NameError:
    pass

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
ds.ReadMode = c.cwdsReadsSynchronous        
ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.SyncRead(-1)

False

In [3]:
if ds.Status == 2:
    ds.SyncRead(10)
    print(repr(ds.Data.Value))
else:
    print(ds.LastMessage)

0


In [4]:
import tkinter as tk
import tkinter.ttk as ttk
from datetime import datetime

In [5]:
class ScrollFrame(tk.Frame):
    def __init__(self, master, height=100, width=100, yscroll=True, xscroll=False):
        super().__init__(master, relief=tk.GROOVE, width=width, height=height)
        self.width = width
        self.height = height
        self.canvas = tk.Canvas(self)
        self.inner_frame = tk.Frame(self.canvas) #, width=width, height=height)
        
        if yscroll:
            self.scrollbar = tk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
            self.canvas.configure(yscrollcommand=self.scrollbar.set)
        else:
            self.scrollbar = None
            
        if xscroll:
            self.xscrollbar = tk.Scrollbar(self, orient="horizontal", command=self.canvas.xview)
            self.canvas.configure(xscrollcommand=self.xscrollbar.set)
        else:
            self.xscrollbar = None
        
        self.window = self.canvas.create_window((0,0), window=self.inner_frame, anchor='nw')
        self.inner_frame.bind("<Configure>", self._on_frame_configure)
        self.canvas.bind("<MouseWheel>", self.on_mouse_scroll)
        self.inner_frame.bind("<MouseWheel>", self.on_mouse_scroll)
        self.items = []
        
    def add_item(self, item):
        self.items.append(item)
    
    def create_widget(self, klass, *args, **kw):
        w = klass(self.inner_frame, *args, **kw)
        self.add_item(w)
        w.bind("<MouseWheel>", self.on_mouse_scroll)
        return w
    
    def _configure_widgets(self):
        
        if self.scrollbar:
            self.scrollbar.pack(side="right", fill=tk.Y)
        if self.xscrollbar:
            self.xscrollbar.pack(side="bottom", fill=tk.X)
        self.canvas.pack(side="left", fill=tk.BOTH)
        #self.canvas.configure(width=self.width-self.scrollbar.winfo_width(), height=self.height-self.xscrollbar.winfo_height())
        #self.inner_frame.pack_configure(fill=tk.X)
        #self.canvas.configure(width=self.width-self.scrollbar.winfo_width())
        for it in self.items:
            it.pack(anchor='w', expand=True, fill=tk.X)
        for ob in self.inner_frame.children.values():
            ob.bind("<MouseWheel>", self.on_mouse_scroll)

    def pack(self, **kw):
        super().pack(**kw)
        self._configure_widgets()
    
    def grid(self, **kw):
        super().grid(**kw)
        self._configure_widgets()            
            
    def _on_frame_configure(self, e):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"), width=self.width, height=self.height)
        
    def on_mouse_scroll(self, e):
        self.canvas.yview_scroll(-int(e.delta/120), "units")


In [7]:
root = tk.Tk()
kvs = {}

try:
    ds.SyncDisconnect(-1)
except NameError:
    pass

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
ds.ReadMode = c.cwdsReadsSynchronous        
ds.SyncConnectTo("dstp://localhost/pbsrioglobals", c.cwdsReadAutoUpdate, -1)
ds.SyncRead(-1)

ts, keys, vals = ds.Data.Value
frame = ScrollFrame(root, 400, 800)
timel = ttk.Label(frame.inner_frame, text = "Time:")
timet = ttk.Label(frame.inner_frame, text=datetime.fromtimestamp(ts).strftime("%H:%M:%S"))

timel.grid(column=0, row=1, sticky=tk.N+tk.W)
timet.grid(column=1, row=1, sticky=tk.N+tk.W)

for i, (key, val) in enumerate(zip(keys, vals), 2):
    l = ttk.Label(frame.inner_frame, text=key)
    t = ttk.Label(frame.inner_frame, text=str(val))
    l.grid(column=0, row=i, sticky=tk.N+tk.W)
    t.grid(column=1, row=i, sticky=tk.N+tk.W)
    kvs[key] = (l, t)
frame.grid()
stop = False
    
lv_offset = datetime(1970, 1, 1) - datetime(1904, 1, 1)
    
def update():
    if ds.Status==2:
        if not ds.SyncRead(0):
            ts, keys, vals = ds.Data.Value
            text = (datetime.fromtimestamp(ts) - lv_offset).strftime("%m/%d/%Y %H:%M:%S")
            timet.config(text=text)
            for k, v in zip(keys, vals):
                kvs[k][1].config(text=repr(type(v))+":"+repr(v))
    else:
        print("Not Connected:", ds.LastMessage)
    if not stop:
        root.after(500, update)
root.after(500, update)

root.mainloop()
stop = True

In [ ]:
oldvalue = -1

In [ ]:
ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

In [ ]:
import matplotlib.pyplot as plt
%matplotlib

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)
line, = ax.plot(ds.Data.Value)
f.show()

In [ ]:
while True:
    ds.SyncRead(-1)
    y = ds.Data.Value
    x = list(range(len(y)))
    line.set_data(x, y)
    f.canvas.flush_events()
    f.canvas.draw()

In [ ]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

Just kidding, this way works too. 

http://zone.ni.com/reference/en-XX/help/375857A-01/html/t_nationalinstruments_net_readmode/
http://zone.ni.com/reference/en-XX/help/375857A-01/xml/usingdatasocketeventsaccessmodesnet/

tldr: in AutoUpdate access mode, the OnDataUpdated event is generated when data is changed. Otherwise, must use Update(). 
In ReadAsynchronous ReadMode, data is read automatically when updated from the server (aka pushed from server to client). Otherwise, muse use SyncRead(). 

TODO: Determine the difference between buffered and unbuffered reads. 

In [ ]:
class EventHandler:
    def OnStatusUpdated(self, *args):
        #print("OnStatusUpdate")
        pass
    def OnDataUpdated(self, ob):
        ob.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)
        print("test")

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
handler = win32com.client.WithEvents(ds, EventHandler)
ds.ReadMode = c.cwdsReadsAsynchronous        
# ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.ConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate)
#ds.SyncRead(-1)
oldvalue = -1

In [ ]:
#ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

In [ ]:
ds.Disconnect()

In [ ]:
dir(lastargs[0])

In [ ]:
lastarg = lastargs[0]
info = lastarg.GetTypeInfo()

In [ ]:
dir(info)

In [ ]:
info.GetNames(1)

In [ ]:
lastarg.Value

In [ ]:
import pythoncom

In [ ]:
pythoncom.DISPATCH_

In [ ]:
lastarg.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)